# UTSA CS 5233: Assignment-1

**Houssein - Awaleh - (llx812)**






## Learning Objectives


*   Read data from a file and Create a graph
*   Implement Uninformed & Informed searching strategies
*   Apply different searching strategies for the given problem
*   Analyze and Compare the searching strategies


## Description

This assignment is focused on **python file reading, graph creation** and implementation of **search algorithms**.
In the following sections, you will complete a series of tasks for a made up problem of *Coronavirus in Texas*.

*   Coronavirus is non-discriminatory, in the sense that it can spread from one city to any other city. The only goal of the virus is to spread to all cities in Texas. Find a possible way for the virus to spread (Uninformed Search).
*   To counter the effect of the virus, vaccine needs to be distributed to all cities. One city has more demand than supply, whereas one city has a shortage of vaccines. The goal is to find an **optimal** strategy to transport the vaccine (Informed Search) from the city with high supply (low demand) to the city with low supply (high demand).

The base structure and comments are provided on what should be done. You can use some libraries that help support you for the successful completion of the assignment. However, you **CANNOT** use a complete library for the search algorithms. You can get pieces of code from online, but please cite the source properly.


#Reading Data Files & Creating a 2D Graph

##(45 points)

In this section, you will write code to read the data files provided, cities.csv and distances.csv, and create a 2D graph consisting of nodes and edges. The goal is to use this graph for the 2 search agents that you will create in the next section.

Provided with this lab, on Blackboard, you will find 2 csv files:

*   **cities.csv** - This file contains a list of coordinates for selected cities in Texas, in the following format:
```
San Antonio,29.4685,-98.5254
```
The above line means that San Antonio is located at the latitude and longitude of 29.4685 N and 98.5254 W respectively. Note that the '-ve' sign denotes 'S' for latitude and 'W' for longitude. While performing calculations you will need to ignore the sign.

*   **distances.csv** - This file contains distance values between two cities of Texas, if a path exists, in the following format:
```
San Antonio,New Braunfels,30.80876734
```
The above line denotes that there should be an edge between *San Antonio* and *New Braunfels* and the weight on that edge, i.e. the distance, is *30.80876734*.

In the code blocks below, handle the logic for the graph. Load the graph data from the give files and display a 2D graph of the given data, with labeled nodes and edges. Create as many functions or code blocks as needed.

##Extra Credit (4 points)

Overlay the 2D graph on an image of the Texas state map.





In [ ]:
# Add only your imports here
import math
from queue import Queue
import sys
import pandas as pd



In [ ]:
# Assume that the data files are in the following folder -- THIS WILL BE USED BY THE TA
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"

file_distance= "/content/sample_data/distances.csv"
file_city = "/content/sample_data/cities.csv"

# Read the distances CSV (no header, so we assign column names)
distances_df = pd.read_csv(file_distance, header=None, names=["City1", "City2", "Distance"])

# Read the cities CSV (no header, so we assign column names)
cities_df = pd.read_csv(file_city, header=None, names=["City", "Latitude", "Longitude"])



In [ ]:
# Load the graph data from the files
class Graph:
    def __init__(self):
        self.nodes =[];
        self.edges = [];
        self.graph ={};
        self.graph_bfs={}
        self.cities ={}


    def add_node(self, node_city):
         self.nodes.append(node_city)


    def load_file_city(self, file_city):
         with open(file_city, "r") as fp:
             for line in fp:

                # read ffrom file city, longitude, latitude

                cityData= line.strip().split(",")

                city = cityData[0].strip();
                lat = cityData[1].strip();
                lon= cityData[2].strip();
                self.cities[city] = ((float(lat), float(lon)))
                self.add_node(city)

    def load_file_city_1(self, cities_df):
        for index, row in cities_df.iterrows():

          city = row['City']
          lat = row['Latitude']
          lon= row['Longitude']
          self.cities[city] = ((float(lat), float(lon)))
          self.add_node(city)



    def add_edges(self, source, destination, distance):
        self.edges.append((source, destination,distance ))


    def load_file_distance(self, file_distance):
        with open(file_distance, "r") as file:
            for line in file:
                data_dist = line.strip().split(",")
                source = data_dist[0].strip()
                destination = data_dist[1].strip();
                distance = float(data_dist[2].strip())
                self.add_edges(source, destination, distance)
                self.add_edge_bfs(source, destination, distance)


    def load_file_distance_1(self, distances_df):
            for index, row in distances_df.iterrows():
              source = row['City1']
              destination = row['City2']
              distance = float(row['Distance'])

              self.add_edges(source, destination, distance)
              self.add_edge_bfs(source, destination, distance)


    def add_edge_bfs(self, city1, city2, distance):
        if city1 not in self.graph_bfs:
             # Initialize dictionary for city1 if it is not present
            self.graph_bfs[city1] = {}
        if city2 not in self.graph_bfs:
            # Initialize dictionary for city2 if it is not present
            self.graph_bfs[city2] = {}
        # Add edges and distance between city1 and city2
        self.graph_bfs[city1][city2] = distance
         # Add edges and distance between city2 and city1
        self.graph_bfs[city2][city1] = distance


    def load_graph(self):

        for edge in self.edges:
            city1, city2, dist = edge
            # graph dictionary takes index city1 or city2 and append all edges
            if city1 not in self.graph:
                self.graph[city1] = []
            #
            if city2 not in self.graph:
                self.graph[city2] = []
            self.graph[city1].append((city2, dist))
            self.graph[city2].append((city1, dist))


    def display_graph(self):

        for city, connections in self.graph.items():
            print(f"{city}: {connections}")

    def bfs_spreading(self, start_city):
    # Set to store visited cities
        visited = set()
            # Create a queue object
        queue = Queue()


        # insert into the queue starting city, path and distance
        queue.put((start_city, [start_city], 0))

        # as long the queue is not empty
        while not queue.empty():
            # pop or dequeue city, path and distance
            city, path, distance = queue.get()

            # if the city is not marked as visited
            if city not in visited:
                visited.add(city)
                print(f"Infecting {city} | Path: {' -> '.join(path)} \n| Distance: {distance} miles \n")


                # Iterate over neighbors edges and their distances
                for edge, edge_distance in self.graph_bfs[city].items():
                    # Check if the neighbor edges is not visited
                    if edge not in visited:
                        # Append the neighbor edges and update the current path
                        update_path = path + [edge]

                        # Add the neighbor's distance to the current distance
                        new_distance = distance + edge_distance

                        # put on the queue neigbor edge, update path and distance
                        queue.put((edge, update_path, new_distance))



    # Haversine formula calculate distance between two coordinate
    # link https://www.movable-type.co.uk/scripts/latlong.html
    def haversine(self, city1, city2):

        lat1, lon1 = self.cities[city1]
        lat2, lon2 = self.cities[city2]
        # Radius of Earth in kilometers
        R = 6371
        # Difference in latitudes in radians
        dLat = math.radians(lat2 - lat1)
        # Difference in longitudes in radians
        dLon = math.radians(lon2 - lon1)
        a = (math.sin(dLat/2) * math.sin(dLat/2) +
            math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
            math.sin(dLon/2) * math.sin(dLon/2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return R * c  # Return the haversine distance in kilometers


    # A* search algorithm
    def a_star_search(self, start, goal):

        # Initialize the edge with the start node
        edge = [(0, start)]
         # Store the parent node for each node
        came_from = {}
        # Store the distance from the start to each node
        distance_so_far = {}
        came_from[start] = None
        distance_so_far[start] = 0

        # Continue until the edge is empty
        while edge:
            # Sort the edges based on distance or cost (f(n) = g(n) + h(n)) highest to lowest
            edge.sort(reverse=True)

            # Pop the node with the lowest cost or distance
            current_cost, current = edge.pop()

            # If the goal is reached break
            if current == goal:
                break

            # check neighbors of the current node
            for neighbor, cost in self.graph.get(current, []):
                # Calculate the new cost for the neighbor
                update_cost = distance_so_far[current] + cost
                # If the neighbor hasn't been visited or has a lower cost, update
                if neighbor not in distance_so_far or update_cost < distance_so_far[neighbor]:
                    distance_so_far[neighbor] = update_cost
                    # Calculate priority with haversine formula
                    priority = update_cost + self.haversine(neighbor, goal)


                    # Append the neighbor to the edges
                    edge.append((priority, neighbor))
                    # Update the parent of the neighbor
                    came_from[neighbor] = current

    # Return the parents and costs for the visited nodes
        return came_from, distance_so_far

    def display_A_star(self, start, goal):
        current = goal
        path = [current]
        came_from, distance_so_far = self.a_star_search(start, goal)
        while current != start:
            current = came_from[current]
            path.append(current)
        #Reverse the path so it goes from start to goal
        path.reverse()
        #Display the result
        print("\nThe path / sequence of cities that will be visited in the optimal vaccine transportation strategy:")
        print( path)
        print("\nThe total distance travelled in the optimal vaccine transportation strategy: ")
        print("Total distance:", distance_so_far[goal], "Kilometers | ", distance_so_far[goal] * 0.621371, "Miles")


In [ ]:
# Display a 2D graph of the given data.

graph = Graph()

graph.load_file_city(file_city)
graph.load_file_distance(file_distance)

# Display a 2D graph of the given data.
print("\t\t\tDisplay a 2D graph of the given data.\n")
graph.load_graph()
graph.display_graph()

			Display a 2D graph of the given data.

San Angelo: [('Midland', 112.2834231), ('Lubbock', 185.1135794), ('Abilene', 95.26906977), ('San Antonio', 210.8494821)]
Midland: [('San Angelo', 112.2834231), ('Lubbock', 117.0903414), ('Odessa', 24.06690284)]
Lubbock: [('San Angelo', 185.1135794), ('Midland', 117.0903414), ('Amarillo', 122.9827931)]
Abilene: [('San Angelo', 95.26906977)]
San Antonio: [('San Angelo', 210.8494821), ('New Braunfels', 30.80876734), ('Seguin', 33.99204585), ('Three Rivers', 74.50909807), ('Uvalde', 82.70993933)]
New Braunfels: [('San Antonio', 30.80876734), ('San Marcos', 18.48948572), ('Seguin', 15.28073104)]
Seguin: [('San Antonio', 33.99204585), ('Gonzalez', 33.4234923), ('New Braunfels', 15.28073104), ('Columbus', 89.60403948)]
Three Rivers: [('San Antonio', 74.50909807), ('Corpus Christi', 81.17701954), ('Alice', 51.26861733), ('Kenedy', 33.25447966)]
Uvalde: [('San Antonio', 82.70993933), ('Del Rio', 70.27570426)]
Austin: [('San Marcos', 30.71742595), ('Roun

#Virus Spread - Uninformed Search Agent

##(40 points)

In this section, you will use the graph created in the previous section and create an *uninformed search* agent that will print the path how the virus will spread to all the provided Texas cities. The first confirmed case of the virus was in **Three Rivers** and starts spreading from there. The virus does not discriminate and it needs to spread to all the cities of Texas.

In the following code block, write code to implement **any** uninformed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path or sequence of cities that will be infected by the spread of Coronavirus.
*   The distance travelled by the selected virus spreading strategy.

##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'

# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'
print("\n\n")
print("\t\t\tSpread of coronavirus from the starting city of 'Three Rivers'\n")
starting = "Three Rivers"
graph.bfs_spreading(starting)





			Spread of coronavirus from the starting city of 'Three Rivers'

Infecting Three Rivers | Path: Three Rivers 
| Distance: 0 miles 

Infecting San Antonio | Path: Three Rivers -> San Antonio 
| Distance: 74.50909807 miles 

Infecting Corpus Christi | Path: Three Rivers -> Corpus Christi 
| Distance: 81.17701954 miles 

Infecting Alice | Path: Three Rivers -> Alice 
| Distance: 51.26861733 miles 

Infecting Kenedy | Path: Three Rivers -> Kenedy 
| Distance: 33.25447966 miles 

Infecting San Angelo | Path: Three Rivers -> San Antonio -> San Angelo 
| Distance: 285.35858017 miles 

Infecting New Braunfels | Path: Three Rivers -> San Antonio -> New Braunfels 
| Distance: 105.31786541 miles 

Infecting Seguin | Path: Three Rivers -> San Antonio -> Seguin 
| Distance: 108.50114391999999 miles 

Infecting Uvalde | Path: Three Rivers -> San Antonio -> Uvalde 
| Distance: 157.2190374 miles 

Infecting Victoria | Path: Three Rivers -> Corpus Christi -> Victoria 
| Distance: 170.2988758999999

#Vaccine Transportation - Informed Search Agent

##(40 points)

In this section, you will create an *informed search* agent that will be used to transport the vaccine. The city of **San Antonio** has more supply of vaccine than the demand. The goal is to create an **optimal strategy** to transport the vaccine and make it available at the highly affected city of **College Station**, where there is a shortage of vaccines.

In the following code block, write code to implement an **optimal** informed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path / sequence of cities that will be visited in the optimal vaccine transportation strategy.
*   The total distance travelled in the optimal vaccine transportation strategy.


##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement an OPTIMAL informed search strategy for distributing the vaccine from 'San Antonio' to 'College Station'
print("\n\n")
print("\t\t\tThe shortest path from San Antonio to College Station\n")

start = "San Antonio" # Starting city
goal = "College Station" # Goal city

#display path and total distance
graph.display_A_star(start, goal)




			The shortest path from San Antonio to College Station


The path / sequence of cities that will be visited in the optimal vaccine transportation strategy:
['San Antonio', 'New Braunfels', 'San Marcos', 'Austin', 'College Station']

The total distance travelled in the optimal vaccine transportation strategy: 
Total distance: 186.77450141 Kilometers |  116.0562587156331 Miles


#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard

##Rubric

*   (45 points) Reading Data files & Creating a 2D Graph
*   (40 points) Virus Spread - Uninformed Search Agent
*   (40 points) Vaccine Transportation - Informed Search Agent
*   (10 points) Extra Credit - on the Texas map image, overlay the 2D graph and the paths selected by the search agents



